In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import pandas as pd
import pickle
from pathlib import Path
from kuma_utils.utils import sigmoid
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score

from datasets import *
from transforms import *
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

In [4]:
test = pd.read_csv('input/g2net-detecting-continuous-gravitational-waves/sample_submission.csv')
public748 = pd.read_csv('input/public748.csv')

In [8]:
np.load('results/ds_09_val1/anomaly_flag.npy')[0].sum()
np.load('results/aug_04/anomaly_flag.npy')[0].sum()

160

In [6]:
model_paths = [
    Path('results/ds_09_val0'),
    Path('results/ds_09_val1'),
    Path('results/aug_04'),
]


In [7]:
prediction_df = {}
oof_df = {}
for p in model_paths:
    if (p/'predictions_tta_wo_anomaly.npy').exists():
        prediction_df[p.stem] = sigmoid(np.load(p/'predictions_tta_wo_anomaly.npy')).reshape(-1)
    else:
        pred = sigmoid(np.load(p/'predictions_tta.npy'))
        oof = sigmoid(np.load(p/'outoffolds.npy'))
        print(pred.shape)
        if pred.shape[1] == 1:
            pred = pred.reshape(-1)
        elif pred.shape[0] == 5:
            pred = pred[0].reshape(-1)
        prediction_df[p.stem] = pred
        oof_df[p.stem] = oof.reshape(-1)
prediction_df = pd.DataFrame(prediction_df)
oof_df = pd.DataFrame(oof_df)
prediction_df['public748'] = public748['target'].values

(5, 7975, 1)
(5, 7975, 1)
(5, 7975, 1)


In [32]:
with open('results/ds_09_val1/folds.pickle', 'rb') as f:
    folds = pickle.load(f)
train = pd.read_csv('input/g2net-detecting-continuous-gravitational-waves/concat_v13_v14_v15.csv')
train = train.iloc[folds[0][1]]
oof_df = oof_df.iloc[folds[0][1]]


In [48]:
roc_auc_score(train.query('25 <= signal_depth')['target'], oof_df.loc[25 <= train['signal_depth']]['ds_09_val0']),\
    roc_auc_score(train.query('25 <= signal_depth')['target'], oof_df.loc[25 <= train['signal_depth']]['ds_09_val1']), \
        roc_auc_score(train.query('25 <= signal_depth')['target'], oof_df.loc[25 <= train['signal_depth']]['aug_04'])

(0.7668152412228258, 0.774607681479373, 0.7740353951145327)

In [12]:
prediction_df

,ds_09,ds_13,aug_01,aug_02,ds_09_mod0,ds_09_mod1,mixup_00,mixup_01,mixup_03,public748,public716
0,0.529081,0.569971,0.366279,0.291880,0.440697,0.538092,0.506956,0.466449,0.752699,0.077132,0.657053
1,0.973470,0.953912,0.984838,0.910586,0.942438,0.997001,0.856452,0.818081,0.889295,0.848641,0.768253
2,0.354265,0.400413,0.289473,0.286450,0.358840,0.439130,0.361939,0.377890,0.764711,0.106932,0.473940
3,0.479609,0.543159,0.314525,0.318305,0.385210,0.408382,0.390988,0.408955,0.739385,0.256063,0.398179
4,0.614418,0.714743,0.644288,0.558036,0.503237,0.567482,0.516554,0.509391,0.785836,0.507310,0.489631
...,...,...,...,...,...,...,...,...,...,...,...
7970,0.999868,0.999971,0.992739,0.999775,0.999169,0.998925,0.988007,0.969016,0.990754,0.937471,0.865461
7971,0.428340,0.475512,0.370228,0.339188,0.375366,0.445702,0.432165,0.423432,0.750777,0.088365,0.585368
7972,0.306392,0.321302,0.228125,0.266770,0.290668,0.354200,0.315550,0.349146,0.696005,0.081671,0.452188
7973,0.360838,0.406303,0.291295,0.344121,0.389977,0.431008,0.460336,0.473722,0.783454,0.112314,0.571307


In [13]:
# test['target'] = prediction_df['aug_04'] * 0.2 + prediction_df['ds_05_aug2'] * 0.2 + prediction_df['public748'] * 0.6
test['target'] = prediction_df['ds_09'] * 0.33 + prediction_df['ds_13'] * 0.33 + prediction_df['public748'] * 0.33
# test['target'] = prediction_df['ds_09_mod0']

In [14]:
test

,id,target
0,00054c878,0.388141
1,0007285a3,0.916088
2,00076c5a6,0.284331
3,001349290,0.422014
4,001a52e92,0.606035
...,...,...
7970,ffbce04ef,0.969312
7971,ffc2d976b,0.327431
7972,ffc905909,0.234091
7973,ffe276f3e,0.290220


In [15]:
test.to_csv('results/blend_submission.csv', index=False)
# test.to_csv(Path('results/ds_09_mod0')/'submission.csv', index=False)